In [1]:

#importing all usefull lib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, train_test_split, KFold, cross_val_predict
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
%matplotlib inline
from sklearn.linear_model import LinearRegression, RidgeCV, Lasso, ElasticNetCV, BayesianRidge, LassoLarsIC
from sklearn.metrics import mean_squared_error, make_scorer 
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
import math

In [2]:
train_dataRaw = pd.read_csv("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode/RawDataSets/train.csv")
test = pd.read_csv("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode/RawDataSets/test.csv")

In [3]:
Sale_Price=train_dataRaw.iloc[:,80]
Sale_Price.shape

(1460,)

In [4]:
train=train_dataRaw.drop(["SalePrice"],axis=1)
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [5]:
data= pd.concat([train,test], keys=['x', 'y'])#here X is training data and Y testing data
data=data.drop(["Id"],axis=1)
data.shape

(2919, 79)

In [6]:
data=data.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
                                       50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
                                       80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
                                       150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
                       "MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                                   7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
                      })

# missing_data

缺失值处理

In [7]:
total_null = data.isnull().sum().sort_values(ascending=False) #First sum and order all null values for each variable
percentage = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False) #Get the percentage
missing_data = pd.concat([total_null, percentage], axis=1, keys=['Total', 'Percentage'])

In [8]:
data = data.drop((missing_data[missing_data["Percentage"] > 0.05]).index,1)

In [9]:
num_col=data._get_numeric_data().columns.tolist()
cat_col=set(data.columns)-set(num_col)

In [10]:
for col in num_col:
    data[col].fillna(data[col].mean(),inplace=True)

In [11]:
for col in cat_col:
    data[col].fillna(data[col].mode()[0],inplace=True)

# Feature Engineering Begins

## 删除highly skewed Data

In [12]:
#delete highly skew category data in numerical variables
train_cat = data[cat_col]
pct=[]
for ix in train_cat.columns:
    temp=train_cat[ix].describe()
    print(temp.describe)
    break
    pct.append(temp["freq"]/temp["count"])
skewData=pd.DataFrame(pct,index=train_cat.columns,columns=["skewness"])
skewData=skewData.sort_values(by="skewness",ascending=False)
print (skewData)
data = data.drop((skewData[skewData['skewness'] >= 0.95]).index,1) 
data.shape

<bound method NDFrame.describe of count     2919
unique       9
top         WD
freq      2526
Name: SaleType, dtype: object>
              skewness
SaleType           NaN
Condition2         NaN
BldgType           NaN
Foundation         NaN
PavedDrive         NaN
RoofStyle          NaN
Functional         NaN
Street             NaN
Condition1         NaN
HeatingQC          NaN
Exterior2nd        NaN
BsmtFinType1       NaN
LandSlope          NaN
Exterior1st        NaN
BsmtExposure       NaN
Neighborhood       NaN
MSZoning           NaN
ExterQual          NaN
BsmtQual           NaN
RoofMatl           NaN
HouseStyle         NaN
Heating            NaN
CentralAir         NaN
Utilities          NaN
LotShape           NaN
LandContour        NaN
ExterCond          NaN
MasVnrType         NaN
MSSubClass         NaN
Electrical         NaN
SaleCondition      NaN
LotConfig          NaN
BsmtCond           NaN
MoSold             NaN
BsmtFinType2       NaN
KitchenQual        NaN


(2919, 68)

In [13]:
train_data=data

In [14]:

#2.2: some categorical features when there is information in the order
#Alley: Type of alley access to property
train_data=train_data.replace({"Alley":{"Grvl" : 1, "Pave" : 2}})
#BsmtCond: Evaluates the general condition of the basement
train_data=train_data.replace({"BsmtCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}})
#BsmtExposure: Refers to walkout or garden level walls
train_data=train_data.replace({"BsmtExposure" : {"No" : 0, "Mn" : 1, "Av": 2, "Gd" : 3}})
#BsmtFinType1: Rating of basement finished area
train_data=train_data.replace({"BsmtFinType1" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6}})
#BsmtFinType2: Rating of basement finished area (if multiple types)
train_data=train_data.replace({"BsmtFinType2" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6}})
#BsmtQual: Evaluates the height of the basement
train_data=train_data.replace({"BsmtQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}})
#ExterCond: Evaluates the present condition of the material on the exterior
train_data=train_data.replace({"ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5}})
#ExterQual: Evaluates the quality of the material on the exterior 
train_data=train_data.replace({"ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5}})
#FireplaceQu: Fireplace quality
train_data=train_data.replace({"FireplaceQu" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}})
#Functional: Home functionality (Assume typical unless deductions are warranted)
train_data=train_data.replace({"Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                       "Min2" : 6, "Min1" : 7, "Typ" : 8}})
#HeatingQC: Heating quality and condition
train_data=train_data.replace({"HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}})
#KitchenQual: Kitchen quality
train_data=train_data.replace({"KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}})
#LandSlope: Slope of property
train_data=train_data.replace({"LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3}})
#LotShape: General shape of property
train_data=train_data.replace({"LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4}})
#PavedDrive: Paved driveway
train_data=train_data.replace({"PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2}})

#2.3: Simplifications of existing categorical 
train_data["Condition1"] = train_data.Condition1.replace({"RRNe" : "Other", 
                                                  "RRNn" : "Other","PosA" : "Other", 
                                                   "RRAe" : "Other"
                                                  })
train_data["Electrical"] = train_data.Electrical.replace({"Mix" : "Other", 
                                                  "FuseP" : "Other"
                                                  })
train_data["Exterior1st"] = train_data.Exterior1st.replace({"AsphShn" : "Other", 
                                                  "CBlock" : "Other","ImStucc" : "Other", 
                                                   "BrkComm" : "Other","Stone" : "Other"
                                                  })
train_data["Exterior2nd"] = train_data.Exterior2nd.replace({"CBlock" : "Other", 
                                                  "AsphShn" : "Other","Stone" : "Other", 
                                                   "Brk Cmn" : "Other","ImStucc" : "Other"
                                                  })
train_data["Foundation"] = train_data.Foundation.replace({"Wood" : "Other", 
                                                  "Stone" : "Other"
                                                  })
# train_data["GarageType"] = train_data.GarageType.replace({"2Types" : "Other", 
#                                                   "CarPort" : "Other"
#                                                   })
train_data["HouseStyle"] = train_data.HouseStyle.replace({"2.5Fin" : "Other", 
                                                  "2.5Unf" : "Other",
                                                  "1.5Unf" : "Other"
                                                  })
train_data["LotConfig"] = train_data.LotConfig.replace({"FR3" : "FR2"
                                                  })
train_data["MSSubClass"] = train_data.MSSubClass.replace({"SC40" : "Other", 
                                                  "SC180" : "Other",
                                                  "SC45" : "Other",
                                                  "SC75" : "Other"
                                                  })
train_data["MSZoning"] = train_data.MSZoning.replace({"C (all)" : "Other", 
                                                  "RH" : "Other"
                                                  })
train_data["Neighborhood"] = train_data.Neighborhood.replace({"Blueste" : "Other", 
                                                  "NPkVill" : "Other",
                                                  "Veenker" : "Other"
                                                  })
train_data["RoofStyle"] = train_data.RoofStyle.replace({"Shed" : "Other", 
                                                  "Mansard" : "Other",
                                                  "Gambrel" : "Other",
                                                  "Flat" : "Other"
                                                  })
train_data["SaleCondition"] = train_data.SaleCondition.replace({"AdjLand" : "Other", 
                                                  "Alloca" : "Other"
                                                  })
train_data["SaleType"] = train_data.SaleType.replace({"Con" : "Other", 
                                                  "Oth" : "Other",
                                                  "CWD" : "Other", "ConLI" : "Other",
                                                  "ConLw" : "Other","ConLD" : "Other"
                                                  })

#2.4 Combinations of existing features
# Overall quality of the house
train_data["OverallGrade"] = train_data["OverallQual"] * train_data["OverallCond"]
# Overall quality of the exterior
train_data["ExterGrade"] = train_data["ExterQual"] * train_data["ExterCond"]
# Overall kitchen score
train_data["KitchenScore"] = train_data["KitchenAbvGr"] * train_data["KitchenQual"]
# Total number of bathrooms
train_data["TotalBath"] = train_data["BsmtFullBath"] + (0.5 * train_data["BsmtHalfBath"]) + \
train_data["FullBath"] + (0.5 * train_data["HalfBath"])
train_data.drop(["BsmtFullBath","BsmtHalfBath","FullBath","HalfBath"],axis=1)
# Total SF for house (incl. basement)
train_data["AllSF"] = train_data["GrLivArea"] + train_data["TotalBsmtSF"]
train_data.drop(["GrLivArea","TotalBsmtSF"],axis=1)
# Total SF for 1st + 2nd floors
train_data["AllFlrsSF"] = train_data["1stFlrSF"] + train_data["2ndFlrSF"]
train_data.drop(["1stFlrSF","2ndFlrSF"],axis=1)
# Total SF for porch
train_data["AllPorchSF"] = train_data["OpenPorchSF"] + train_data["EnclosedPorch"] + \
train_data["3SsnPorch"] + train_data["ScreenPorch"]
train_data.drop(["OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch"],axis=1)
# House completed before sale or not
train_data["BoughtOffPlan"] = train_data.SaleCondition.replace({"Abnorml" : 0, "Alloca" : 0, "AdjLand" : 0, 
                                                      "Family" : 0, "Normal" : 0, "Partial" : 1})

In [15]:
# train_data["OverallQual-s2"] = train_data["OverallQual"] ** 2
# train_data["AllSF-2"] = train_data["AllSF"] ** 2
# train_data["AllFlrsSF-2"] = train_data["AllFlrsSF"] ** 2
# train_data["GrLivArea-2"] = train_data["GrLivArea"] ** 2
# # train_data["SimplOverallQual-s2"] = train_data["SimplOverallQual"] ** 2
# train_data["ExterQual-2"] = train_data["ExterQual"] ** 2
# train_data["GarageCars-2"] = train_data["GarageCars"] ** 2
# train_data["TotalBath-2"] = train_data["TotalBath"] ** 2
# train_data["KitchenQual-2"] = train_data["KitchenQual"] ** 2

In [16]:
# bjadsfk=

## droping some unnecessary cat_features bcoz they have 80% + same value and 20% - defertnt values so they can't effect score【存疑】

In [17]:
data=train_data

In [18]:

df=data.drop(["RoofMatl","Heating","Condition2","BsmtCond","SaleType",
                "CentralAir","Functional","Electrical","LandSlope","ExterCond","Condition1",
               "GarageArea","BsmtUnfSF","3SsnPorch","MiscVal"],axis=1)
# new   


In [19]:
corrmat = train_dataRaw.corr()
top_corr_features = corrmat.index[abs(corrmat["SalePrice"])>0.5]

In [20]:
df=df.drop(["BldgType","Exterior1st","Exterior2nd","Foundation","BsmtFinType2","YrSold","SaleCondition","RoofStyle"],axis=1)

In [21]:
df.shape

(2919, 53)

# END OF FEATURE ENGINEERING

In [22]:
X=pd.get_dummies(df)
X.shape

(2919, 119)

In [23]:
scalerX = MinMaxScaler(feature_range=(0, 1))
X[X.columns] = scalerX.fit_transform(X[X.columns])

In [24]:
#Training data after preproscing

Train_data=X.loc["x"]
Train_data.shape

(1460, 119)

In [25]:
#Testing data after preproscing
Test_data=X.loc["y"]
Test_data.shape

(1459, 119)

In [26]:
#here we add salePrice column in traning data

Train_data.insert(2,column="SalePrice",value=Sale_Price)
Train_data.head()

,LotArea,LotShape,SalePrice,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,BsmtQual,...,MoSold_Jul,MoSold_Jun,MoSold_Mar,MoSold_May,MoSold_Nov,MoSold_Oct,MoSold_Sep,BoughtOffPlan_0,BoughtOffPlan_1,BoughtOffPlan_Other
0,0.033420,1.000000,208500,0.666667,0.500,0.949275,0.883333,0.12250,0.666667,0.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.038795,1.000000,181500,0.555556,0.875,0.753623,0.433333,0.00000,0.333333,0.666667,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.046507,0.666667,223500,0.666667,0.500,0.934783,0.866667,0.10125,0.666667,0.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,0.038561,0.666667,140000,0.666667,0.500,0.311594,0.333333,0.00000,0.333333,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.060576,0.666667,250000,0.777778,0.500,0.927536,0.833333,0.21875,0.666667,0.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [27]:
x=Train_data.drop(["SalePrice"],axis=True)
y=Train_data["SalePrice"]

In [28]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.30,random_state=40)

In [29]:
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor(n_estimators = 50,random_state=40,
                          min_impurity_decrease=0.002,min_weight_fraction_leaf=0.001,min_samples_split=5)
rfr.fit(x_train,y_train)
y_predictrfc = rfr.predict(x_test)

#here we can check our model score
print(rfr.score(x_test,y_test))

0.8836571851763504


In [30]:
rmse = math.sqrt(mean_squared_error(y_test, rfr.predict(x_test)))

print("mear squares error :",rmse)

mear squares error : 24612.11305682439


In [31]:
from sklearn.tree import DecisionTreeRegressor
dtr=DecisionTreeRegressor(random_state=140,min_samples_split=5,min_impurity_decrease=0.002,min_weight_fraction_leaf=0.001)
dtr.fit(x_train,y_train)

#u can also use GridSearchCV / random Searchcv for hyperperameter tuning
print(dtr.score(x_test,y_test))


0.6483165194454803


In [32]:
rmse = math.sqrt(mean_squared_error(y_test, dtr.predict(x_test)))
print("RMSE:",rmse)

RMSE: 42791.26273260494


In [33]:
GBoost = GradientBoostingRegressor(n_estimators=5000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
#RMSE estimated through the partition of the train set
GBoost.fit(x_train, y_train)

print("RMSE: %.4f" % rmse)

RMSE: 42791.2627


In [34]:
print(GBoost.score(x_test,y_test))

0.9170522779492055


In [35]:
#here we see test data here one column is missing that is Saleprice bcoz that is need to predict
Test_data.head()

,LotArea,LotShape,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,BsmtQual,BsmtExposure,...,MoSold_Jul,MoSold_Jun,MoSold_Mar,MoSold_May,MoSold_Nov,MoSold_Oct,MoSold_Sep,BoughtOffPlan_0,BoughtOffPlan_1,BoughtOffPlan_Other
0,0.048246,1.000000,0.444444,0.625,0.644928,0.183333,0.0000,0.333333,0.333333,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.060609,0.666667,0.555556,0.625,0.623188,0.133333,0.0675,0.333333,0.333333,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.058566,0.666667,0.444444,0.500,0.905797,0.800000,0.0000,0.333333,0.666667,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.040562,0.666667,0.555556,0.625,0.913043,0.800000,0.0125,0.333333,0.333333,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.017318,0.666667,0.777778,0.500,0.869565,0.700000,0.0000,0.666667,0.666667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [36]:
Test_data.shape

(1459, 119)

In [37]:
y_model_prerfc = GBoost.predict(Test_data)
# y_model_prerfc=dtr.predict(Test_data)

In [38]:
y_model_prerfc=np.around(y_model_prerfc,0)

In [39]:
prediction=np.array(y_model_prerfc).tolist()
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [40]:
test.insert(1,column="SalePrice",value=prediction)
test.head()

,Id,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,114764.0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,164287.0,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,180058.0,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,199624.0,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,181867.0,120,RL,43.0,5005,Pave,NaN,IR1,HLS,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [41]:
predict_sub=test.drop(test.iloc[:,2:],axis=1)
predict_sub.head()

,Id,SalePrice
0,1461,114764.0
1,1462,164287.0
2,1463,180058.0
3,1464,199624.0
4,1465,181867.0


In [42]:
predict_sub.shape


(1459, 2)

In [43]:
predict_sub.to_csv('Home_predictionsGB.csv',index=False)